In [ ]:
import socket
import time
import threading
from jetracer.nvidia_racecar import NvidiaRacecar

# UDP server configuration for Robot 2
SERVER_IP = '0.0.0.0'  # Listen on all available interfaces
SERVER_PORT = 5057     # Port to listen on
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.bind((SERVER_IP, SERVER_PORT))

print(f"Listening for messages on {SERVER_IP}:{SERVER_PORT}")

# Initialize the car
car = NvidiaRacecar()

# Increase throttle gain for more power
car.throttle_gain = 1.0  # Adjust as needed

# Global variable to keep track of position
position = (0, 0)
stop_navigation = threading.Event()  # Event to signal navigation to stop

# Function to move the car forward
def move_forward(throttle=-0.8):  # Increase throttle
    print("Moving forward with increased throttle...")
    car.throttle = throttle

# Function to stop the car
def stop():
    print("Stopping the car...")
    car.throttle = 0

# Function to navigate the robot using dead reckoning
def navigate():
    global position
    speed = 75  # speed of the robot in cm/s
    segment_length = 30  # length of each segment in cm
    time_per_segment = segment_length / speed  # time to travel one segment

    x, y = position
    print(f"Starting navigation from position {position}...")

    move_forward()  # Start moving the car

    try:
        while not stop_navigation.is_set():
            time.sleep(time_per_segment)  # wait for the time to travel one segment
            x += 1  # move to the next segment
            current_time = round(time_per_segment * x, 2)  # total time elapsed

            position = (x, y)
            print(f"Updated position: {position}, Time elapsed: {current_time}s")

            if x >= 6:  # Optional: Stop after reaching coordinate (6,0)
                break
    except KeyboardInterrupt:
        stop()  # Stop the car when interrupted
        print("Navigation stopped.")
    finally:
        stop()  # Ensure car stops if navigation ends
        print(f"Final position: {position}")

# Function to start thermal camera and receive data
def receive_thermal_sensor():
    global stop_navigation
    # Setup for sending command
    COMMAND_IP = "192.168.1.107"  # IP address of your Raspberry Pi
    COMMAND_PORT = 5100
    command_sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    
    # Setup for receiving thermal sensor data
    UDP_IP = "0.0.0.0"  # Listen on all available interfaces
    UDP_PORT = 5101
    thermal_sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    thermal_sock.bind((UDP_IP, UDP_PORT))
    
    # Send start command to Raspberry Pi
    start_command = "start_thermal_camera"
    command_sock.sendto(start_command.encode(), (COMMAND_IP, COMMAND_PORT))
    print(f"Sent command: {start_command} to {COMMAND_IP}:{COMMAND_PORT}")

    print(f"Listening for thermal sensor data on {UDP_IP}:{UDP_PORT}")

    try:
        while not stop_navigation.is_set():
            #data, addr = thermal_sock.recvfrom(1024)  # Buffer size is 1024 bytes
            #highest_temp = float(data.decode())
            
            # Check if the highest temperature exceeds 30°C
            time.sleep(1) 
            highest_temp = 35
            
            if highest_temp > 30:
                print(f"Received highest temperature: {highest_temp}°C")
                stop_navigation.set()  # Signal navigation to stop

                # Send the detected temperature and position
                Possible_Human(position)
                break
    except KeyboardInterrupt:
        print("Thermal sensor data reception stopped.")
    finally:
        command_sock.close()
        thermal_sock.close()

# Function to handle possible human detection
def Possible_Human(coordinates):
    print("Possible human detected! Sending coordinates...")
    message = "possible human detection"
    coord_message = f"{coordinates[0]},{coordinates[1]}"

    # UDP client configuration to send message to the other robot
    UDP_IP = "172.20.10.3"  # IP address of the other robot 192.168.1.109 'Home wifi'
    UDP_PORT = 5076
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

    # Send "possible human detection" message
    sock.sendto(message.encode(), (UDP_IP, UDP_PORT))
    time.sleep(0.5)  # Small delay to ensure messages are sent separately

    # Send coordinates message
    sock.sendto(coord_message.encode(), (UDP_IP, UDP_PORT))
    print(f"Sent coordinates: {coordinates} to {UDP_IP}:{UDP_PORT}")

# Function to start operations upon receiving an emergency message
def start_operations():
    print("Executing emergency operations...")
    
    # Create threads for navigation and thermal camera operations
    navigation_thread = threading.Thread(target=navigate)
    thermal_thread = threading.Thread(target=receive_thermal_sensor)
    
    # Start both threads
    navigation_thread.start()
    thermal_thread.start()

# Main loop to listen for messages from Robot 1
try:
    while True:
        message, address = sock.recvfrom(1024)  # Buffer size is 1024 bytes
        message = message.decode('utf-8')  # Decode the message from bytes to string

        print(f"Received message: '{message}' from {address}")

        if message == "EMERGENCY":
            print("Emergency detected! Starting operations...")
            start_operations()
except KeyboardInterrupt:
    print("Server interrupted. Shutting down.")
finally:
    sock.close()  # Ensure socket is closed properly


Listening for messages on 0.0.0.0:5057
Received message: 'EMERGENCY' from ('172.20.10.3', 52781)
Emergency detected! Starting operations...
Executing emergency operations...
Starting navigation from position (0, 0)...
Moving forward with increased throttle...
Sent command: start_thermal_camera to 192.168.1.107:5100
Listening for thermal sensor data on 0.0.0.0:5101
Updated position: (1, 0), Time elapsed: 0.4s
Updated position: (2, 0), Time elapsed: 0.8s
Received highest temperature: 35°C
Possible human detected! Sending coordinates...
Updated position: (3, 0), Time elapsed: 1.2s
Stopping the car...
Final position: (3, 0)
Sent coordinates: (2, 0) to 172.20.10.3:5076
